# TDC - bbb-martins dataset curation

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm

root = pathlib.Path("__file__").absolute().parents[2]
# set to recipe root directory
os.chdir(root)
sys.path.insert(0, str(root))

In [ ]:
org = "polaris"
data_name = "bbb-martins"
dirname = dm.fs.join(root, f"org-{org}/tdc", data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/tdc/{data_name}"

/mnt/ps/home/CORP/lu.zhu/miniconda3/envs/po_datasets/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the raw data
The dataset is published in https://pubmed.ncbi.nlm.nih.gov/26501955

In [ ]:
# Load the data
from tdc.single_pred import ADME
data = ADME(name = 'BBB_Martins')
split = data.get_split()
split.keys()

Found local copy...
Loading...
Done!


dict_keys(['train', 'valid', 'test'])

In [ ]:
# concate the splits into one table for curation step
data = pd.DataFrame()
for split_key in split.keys():
    split_df = split[split_key]
    split_df['split'] = split_key
    data = pd.concat([data, split_df], ignore_index=True)

data


,Drug_ID,Drug,Y,split
0,Terbutylchlorambucil,CC(C)(C)OC(=O)CCCc1ccc(N(CCCl)CCCl)cc1,1,train
1,40730,CC1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23,1,train
2,cloxacillin,Cc1onc(-c2ccccc2Cl)c1C(=O)N[C@@H]1C(=O)N2[C@@H...,1,train
3,cefoperazone,CCN1CCN(C(=O)N[C@@H](C(=O)N[C@@H]2C(=O)N3C(C(=...,1,train
4,rolitetracycline,CN(C)[C@@H]1C(=O)/C(=C(/O)NCN2CCCC2)C(=O)[C@@]...,1,train
...,...,...,...,...
2025,rifamide,CCN(CC)C(=O)COc1cc2c(O)c3c(O)c(C)c4c(c13)C(=O)...,0,test
2026,BRL52871,O=C(Cc1ccc(Cl)c(Cl)c1)N1CCc2sccc2[C@H]1CN1CCCC1,1,test
2027,hydromorphone,CN1CC[C@]23c4c5ccc(O)c4O[C@H]2C(=O)CC[C@H]3[C@...,1,test
2028,suritozole,Cn1nc(-c2cccc(F)c2)n(C)c1=S,1,test


In [ ]:
data.split.value_counts()

split
train    1421
test      406
valid     203
Name: count, dtype: int64

In [ ]:
# Define key columns for curation
mol_col = 'Drug'
data_cols = ['Y']

In [ ]:
# import key curation components from auroris
from auroris.curation import Curator
from auroris.curation.actions import (
    MoleculeCuration,
    OutlierDetection,
    Deduplication,
    StereoIsomerACDetection,
    Discretization,
)

# Define the curation workflow
curator = Curator(
    steps=[
        MoleculeCuration(input_column=mol_col, y_cols=data_cols),
        StereoIsomerACDetection(y_cols=data_cols),
    ],
    parallelized_kwargs={"n_jobs": -1},
)

curator.to_json(f"{dirname}/inspection_config.json")

In [ ]:
# Run the curation step defined as above
data_inspection, report = curator(data)

2024-07-29 09:10:43.286 | INFO     | auroris.curation._curator:transform:106 - Performing step: mol_curation


In [ ]:
#  get the curation logger
from auroris.report.broadcaster import LoggerBroadcaster

broadcaster = LoggerBroadcaster(report)
broadcaster.broadcast()

NameError: name 'report' is not defined

In [ ]:
# Generate an HTML report with embedded visualizations showcasing the data analysis.
from utils.auroris_utils import HTMLBroadcaster

# export report to local directory
broadcaster = HTMLBroadcaster(report, f"{dirname}/inspection_report")
report_path = broadcaster.broadcast()

In [ ]:
# check the curated data
data_inspection.describe(include="all")

,Smiles,EGFR_L858R,EGFR,KIT_T6701,KIT_V560G,KIT,RET_V804L,RET_Y791F,RET,MOL_smiles,...,OUTLIER_RET_Y791F,OUTLIER_RET,AC_CLS_EGFR_L858R,AC_CLS_EGFR,AC_CLS_KIT_T6701,AC_CLS_KIT_V560G,AC_CLS_KIT,AC_CLS_RET_V804L,AC_CLS_RET_Y791F,AC_CLS_RET
count,366,366.000000,365.000000,366.000000,366.000000,365.000000,366.000000,365.000000,365.000000,366,...,366,366,366,366,366,366,366,366,366,366
unique,366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,364,...,1,1,1,1,1,1,1,1,2,2
top,C#Cc1cccc(Nc2ncnc3ccc(OCCCCNCCS(C)(=O)=O)cc23)c1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nc1ncnc2occ(-c3ccc(NC(=O)Nc4cc(C(F)(F)F)ccc4F)...,...,False,False,False,False,False,False,False,False,False,False
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,...,366,366,366,366,366,366,366,366,362,364
mean,NaN,19.306325,16.719288,13.226175,12.752158,26.747178,18.211011,24.460123,21.998904,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,27.276753,26.484272,24.209365,23.535177,33.886105,22.263974,24.631992,25.271774,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,-5.940000,-20.030000,-3.320000,-3.930000,-26.190000,-3.850000,-4.940000,-1.840000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,2.642500,2.620000,1.247500,0.340000,3.600000,4.402500,7.110000,5.380000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,6.425000,5.530000,3.630000,2.950000,8.760000,8.985000,13.890000,9.600000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,22.092500,11.920000,7.360000,10.367500,45.850000,20.637500,33.580000,28.980000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Check the activity shift between the stereo isomers


![ac_stereo_CLS_RET_Y791F](inspection_report/images/18-Activity_shifts_among_stereoisomers__CLS_RET_Y791F.png)


![ac_stereo_CLS_RET](inspection_report/images/19-Activity_shifts_among_stereoisomers__CLS_RET.png)


In [ ]:
# Check the stereoisomers
data.loc[[136, 137, 302, 309], [col for col in data.columns if "RET" in col]]

,RET_V804L,RET_Y791F,RET
136,38.26,72.03,72.10
137,54.75,63.76,51.82
302,73.94,68.99,71.26
309,78.31,72.37,79.90


With the predefined threshold value `>70`, a few of molecules has been detected having activity shift between the isomer for `RET_(Y791F_mutant)`, and `RET`. 
For the stereoisomers #136 and #137, the activity value on target `RET_(Y791F_mutant)` are very close, while there is a significant difference for target `RET`. The stereocenters of the pair of stereoisomers are well annotated.


For the second pair (#302 and #309) - the double bond isomers, the stereo information is not specified for the first molecule(#309). Given the activity values are similar. Therefore, we consider keeping the molecule #302 for the target `RET_(Y791F_mutant)`.

In [ ]:
# mask clf value for #309
data.loc[309, "RET_Y791F"] = None
# Mask clf value for #136, #137
data.loc[[136, 137], ["RET", "RET_Y791F"]] = None

## Rerun data curation and export curated data for downstream tasks


### Rerun the curation steps

In [ ]:
curator = Curator(
    data_path=source_data_path,
    steps=[
        MoleculeCuration(input_column=mol_col, y_cols=data_cols),
        Deduplication(
            deduplicate_on=mol_col, y_cols=data_cols
        ),  # remove the replicated molecules
        Discretization(input_column="EGFR", thresholds=[75], label_order="ascending"),
        Discretization(
            input_column="EGFR_L858R", thresholds=[75], label_order="ascending"
        ),
        Discretization(input_column="KIT", thresholds=[75], label_order="ascending"),
        Discretization(
            input_column="KIT_T6701", thresholds=[75], label_order="ascending"
        ),
        Discretization(
            input_column="KIT_V560G", thresholds=[75], label_order="ascending"
        ),
        Discretization(input_column="RET", thresholds=[70], label_order="ascending"),
        Discretization(
            input_column="RET_V804L", thresholds=[70], label_order="ascending"
        ),
        Discretization(
            input_column="RET_Y791F", thresholds=[70], label_order="ascending"
        ),
        OutlierDetection(
            method="zscore", columns=data_cols, threshold=3, use_modified_zscore=True
        ),
        StereoIsomerACDetection(y_cols=[f"CLS_{col}" for col in data_cols]),
    ],
    parallelized_kwargs={"n_jobs": -1},
)

# The final curation configuration is exported for reproducibility
path = f"{gcp_root}/data/curation/curation_config.json"
curator.to_json(path)

In [ ]:
# Run the curation step defined as above
data_curated, report = curator(data)

2024-07-10 02:01:39.139 | INFO     | auroris.curation._curator:transform:106 - Performing step: mol_curation
2024-07-10 02:01:41.337 | INFO     | auroris.curation._curator:transform:106 - Performing step: deduplicate
2024-07-10 02:01:41.549 | INFO     | auroris.curation._curator:transform:106 - Performing step: discretize
2024-07-10 02:01:41.603 | INFO     | auroris.curation._curator:transform:106 - Performing step: discretize
2024-07-10 02:01:41.655 | INFO     | auroris.curation._curator:transform:106 - Performing step: discretize
2024-07-10 02:01:41.705 | INFO     | auroris.curation._curator:transform:106 - Performing step: discretize
2024-07-10 02:01:41.857 | INFO     | auroris.curation._curator:transform:106 - Performing step: discretize
2024-07-10 02:01:41.913 | INFO     | auroris.curation._curator:transform:106 - Performing step: discretize
2024-07-10 02:01:41.966 | INFO     | auroris.curation._curator:transform:106 - Performing step: discretize
2024-07-10 02:01:42.017 | INFO    

In [ ]:
# Export report to polaris public directory on GCP
# The report is ready to reviewed in the HTML file.
broadcaster = HTMLBroadcaster(
    report, f"{gcp_root}/data/curation/report", embed_images=True
)
broadcaster.broadcast()

'gs://polaris-public/polaris-recipes/org-polaris/drewry2014_pkis1_subset/data/curation/report/index.html'

In [ ]:
broadcaster = LoggerBroadcaster(report)
broadcaster.broadcast()

===== Curation Report =====
Time: 2024-07-10 02:01:39
Version: dev
===== mol_curation =====
[LOG]: New column added: MOL_smiles
[LOG]: New column added: MOL_molhash_id
[LOG]: New column added: MOL_molhash_id_no_stereo
[LOG]: New column added: MOL_num_stereoisomers
[LOG]: New column added: MOL_num_undefined_stereoisomers
[LOG]: New column added: MOL_num_defined_stereo_center
[LOG]: New column added: MOL_num_undefined_stereo_center
[LOG]: New column added: MOL_num_stereo_center
[LOG]: New column added: MOL_undefined_E_D
[LOG]: New column added: MOL_undefined_E/Z
[LOG]: Default `ecfp` fingerprint is used to visualize the chemical space.
[LOG]: Molecules with undefined stereocenter detected: 9.
[IMG]: Dimensions 2400 x 2400
[IMG]: Dimensions 1200 x 900
===== deduplicate =====
[LOG]: Deduplication merged and removed 0 duplicated molecules from dataset
===== discretize =====
[LOG]: New column added: CLS_EGFR
[IMG]: Dimensions 1200 x 600
===== discretize =====
[LOG]: New column added: CLS_EGF

## Export the final curated data

In [ ]:
fout = f"{gcp_root}/data/curation/{data_name}_curated.csv"
data_curated.reset_index(drop=True).to_csv(fout, index=False)

In [ ]:
fout

'gs://polaris-public/polaris-recipes/org-polaris/drewry2014_pkis1_subset/data/curation/drewry2014_pkis1_subset_curated.csv'